In [1]:
import mcdm
import numpy as np
from pymcdm.methods import TOPSIS
from pymcdm.methods import VIKOR
from pymcdm.methods import PROMETHEE_II
from pymcdm.helpers import rankdata, rrankdata
from pymcdm import weights as mcdm_weights

In [2]:
from prettytable import from_csv
with open("heart_perform.tsv") as fp:
    mytable = from_csv(fp)
print(mytable)

+----------------------+----------+-----------+--------+-------------+----------+-------------+-------------+-------------+--------------+--------------+
|                      | ACCURACY | PRECISION | RECALL |     AUC     | F1-SCORE |   FP_RATE   |   FN_RATE   | SPECIFICITY | HAMMING LOSS | ENTROPY LOSS |
+----------------------+----------+-----------+--------+-------------+----------+-------------+-------------+-------------+--------------+--------------+
| LOGISTIC REGRESSION  |   0.8    |    0.86   |  0.8   | 0.949891068 |   0.8    | 0.352941176 |      0      | 0.647058824 | 0.196721311  | 6.794670687  |
|    DECISION TREE     |   0.74   |    0.76   |  0.74  | 0.749455338 |   0.74   | 0.352941176 | 0.148148148 | 0.647058824 | 0.262295082  | 9.059508483  |
| GAUSSIAN NAIVE BAYES |   0.87   |    0.9    |  0.87  | 0.934640523 |   0.87   | 0.235294118 |      0      | 0.764705882 | 0.131147541  | 4.529780458  |
|         SVM          |   0.44   |    0.2    |  0.44  | 0.566448802 |   0.2

In [5]:
x_matrix, alt_names = mcdm.load("heart_perform.tsv", delimiter="\t", skiprows=1, labeled_rows=True)
alts = np.array(x_matrix, dtype='float')
weights = mcdm_weights.equal(alts)
types = np.array([1,1,1,1,1,-1,-1,1,-1,-1])

In [6]:
topsis = TOPSIS()
# Determine preferences and ranking for alternatives
pref1 = topsis(alts, weights, types)
ranking1= rrankdata(pref1)

for r, p in zip(ranking1, pref1):
    print(r, p)


3.0 0.8173925466965153
4.0 0.7125559408083195
1.0 0.9004778598552923
7.0 0.26775097629314826
6.0 0.3979146596442731
2.0 0.8178078602327319
5.0 0.6305448503164852


In [7]:
vikor= VIKOR()
pref2 = vikor(alts, weights, types)
ranking2= rankdata(pref2)

for r, p in zip(ranking2, pref2):
    print(r, p)

3.0 0.13929974892437708
4.0 0.22000182332886453
1.0 0.0
7.0 1.0
6.0 0.8701588157885123
2.0 0.11238057353242026
5.0 0.3183894935597914


In [8]:
promethe=PROMETHEE_II('usual')
p = np.random.rand(x_matrix.shape[1]) / 2
q = np.random.rand(x_matrix.shape[1]) / 2 + 0.5
pref3 = promethe(alts, weights, types,p=p,q=q)
ranking3 = rrankdata(pref3)

for r, p in zip(ranking3, pref3):
    print(r, p)

2.0 0.4666666666666667
4.0 -0.06666666666666676
1.0 0.8666666666666665
7.0 -0.7999999999999998
5.0 -0.4000000000000001
3.0 0.3666666666666666
6.0 -0.4333333333333334


In [9]:
from prettytable import PrettyTable
print("#########################PIMA DATASET#######################")
columns = ["Model", "TOPSIS","Value","VIKOR","VALUES","PROMETHEE","VALUES"]
Table = PrettyTable()
Table.add_column(columns[0], ["LOGISTIC REGRESSION ", "DECISION TREE", "GAUSSIAN NAIVE BAYES",
                       "SVM ", "NEURAL NETWORK", "RANDOM FOREST ", "LINEAR MODEL"])
Table.add_column(columns[1], ranking1)
Table.add_column(columns[2], pref1)
Table.add_column(columns[3], ranking2)
Table.add_column(columns[4], pref2)
Table.add_column(columns[5], ranking3)
Table.add_column(columns[6], pref3)

print(Table)

#########################PIMA DATASET#######################
+----------------------+--------+---------------------+-------+---------------------+-----------+----------------------+
|        Model         | TOPSIS |        Value        | VIKOR |        VALUES       | PROMETHEE |        VALUES        |
+----------------------+--------+---------------------+-------+---------------------+-----------+----------------------+
| LOGISTIC REGRESSION  |  3.0   |  0.8173925466965153 |  3.0  | 0.13929974892437708 |    2.0    |  0.4666666666666667  |
|    DECISION TREE     |  4.0   |  0.7125559408083195 |  4.0  | 0.22000182332886453 |    4.0    | -0.06666666666666676 |
| GAUSSIAN NAIVE BAYES |  1.0   |  0.9004778598552923 |  1.0  |         0.0         |    1.0    |  0.8666666666666665  |
|         SVM          |  7.0   | 0.26775097629314826 |  7.0  |         1.0         |    7.0    | -0.7999999999999998  |
|    NEURAL NETWORK    |  6.0   |  0.3979146596442731 |  6.0  |  0.8701588157885123 |    5.0

In [10]:
ans=mcdm.rank(x_matrix, alt_names=alt_names, n_method="Linear2", w_method="CRITIC", s_method="SAW") 
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################SAW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table) 

#########################SAW#######################
+----------------------+--------------------+
|        Model         |       Value        |
+----------------------+--------------------+
|    RANDOM FOREST     | 0.5222199941637795 |
| LOGISTIC REGRESSION  | 0.5189854598686782 |
| GAUSSIAN NAIVE BAYES | 0.5144364840403131 |
|    DECISION TREE     | 0.5025773303267663 |
|     LINEAR MODEL     | 0.5022365320306864 |
|    NEURAL NETWORK    | 0.4693691330513972 |
|         SVM          | 0.4038843035670069 |
+----------------------+--------------------+


In [11]:
ans=mcdm.rank(x_matrix,alt_names=alt_names,n_method="Linear1",w_method="CRITIC",s_method="MEW")
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################MEW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table)

#########################MEW#######################
+----------------------+---------------------+
|        Model         |        Value        |
+----------------------+---------------------+
|     LINEAR MODEL     |  0.5228061030949025 |
|    DECISION TREE     | 0.47842361251955107 |
|    RANDOM FOREST     | 0.41480742326706527 |
| LOGISTIC REGRESSION  |         0.0         |
| GAUSSIAN NAIVE BAYES |         0.0         |
|         SVM          |         0.0         |
|    NEURAL NETWORK    |         0.0         |
+----------------------+---------------------+
